<a href="https://colab.research.google.com/github/ffer200395/Scraping-financial-data/blob/main/Roulette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To generate random sequences
from numpy import random
import numpy as np

# For data handling
import pandas as pd
from itertools import groupby

from tqdm import tqdm

# Function for double and percentage strategy:

In [ ]:
def double_percentage_strategy(runs,color,percentage=None):
  # Initially we bet 1€ so money spent in bets is 1 
  bet = 1  
  # To save data regarding bets and wins
  lbet, lwin = [bet], []
  # For every spin
  for i,run in enumerate(runs):
    # If winner color you win double your bet and next bet is 1 else you win 0€ and double your bet
    win, bet = (bet*2, 1) if run==color else (0, bet*2)
    lwin.append(win)
    # Calculate final profit and max bet done
    realWin=sum(lwin)-sum(lbet) 
    #print(lwin, lbet, realWin)
    # If we play until a predifined moment (given by percentage) and we already obtained profit
    if (percentage!=None) and (i>=int(len(runs)*percentage)) and (realWin>0):
      # Exit the game
      break 
    # Save data
    if i!=len(runs)-1:
      lbet.append(bet)
  return realWin,max(lbet)

# Examples for double_percentage_strategy:

In this example we see we bet to 1 and one appears 3 times so final profit is 3€ and maximum amount bet was 1€:

In [ ]:
# Example always guess the color in all runs (1 for red, 2 for black)
double_percentage_strategy([1,1,1,1], 1)

(4, 1)

In [ ]:
# Example always lose the color in all runs (1 for red, 2 for black)
double_percentage_strategy([0,2,0,2], 1)

[0] [1]
[0, 0] [1, 2]
[0, 0, 0] [1, 2, 4]
[0, 0, 0, 0] [1, 2, 4, 8]


(-15, 8)

Since we lost n=4 consecutive times our maximum bet was 8€ (2^(n-1)) and we would have needed in our wallet in our wallet 15€ ((2^n)-1).

In [ ]:
(2**4)-1

15

We can consider quiting earlier if we already made some profit and have played a certain percentage of the total number of runs the user wants to play (double percentage strategy):

In [ ]:
# Example apply double percentage strategy over a player that plans two play 10 
# runs but if he already played 5 and has profit he is willing to quit
double_percentage_strategy([0,1,1,1,0,2,1,1,1,1], 1, 0.5)

(4, 4)

In [ ]:
# Same sequence of runs whith lower,higher and no percentage
print(double_percentage_strategy([0,1,1,1,0,2,1,1,1,1], 1, 0.2))
print(double_percentage_strategy([0,1,1,1,0,2,1,1,1,1], 1, 0.8))
print(double_percentage_strategy([0,1,1,1,0,2,1,1,1,1], 1))

(2, 2)
(6, 4)
(7, 4)


# Function for simulations:

In [ ]:
def simulate(rounds, runs, percentage=None):
  # DataFrame to store data
  res=pd.DataFrame(columns=['Round_num','WinR','WinB','MaxBetR','MaxBetB'])
  # In every experiment data is gathered for both colors
  for round in tqdm(range(rounds)):
    # Generate a random sequence of 0,1,2 (Zero, Red, Black)
    all_runs = [random.choice(np.arange(0,3), p=[0.02702, 0.48649, 0.48649]) for _ in range(runs)]
    # Results of applying the strategy over red and blue color
    r, b = double_percentage_strategy(all_runs,1,percentage), double_percentage_strategy(all_runs,2,percentage)
    # Retrurn generated dataset
    res=res.append({'Round_num':round,'WinR':r[0],'WinB':b[0],'MaxBetR':r[1],'MaxBetB':b[1]}, ignore_index=True)
  return res

We now simulate 10.000 gamblers playing 100 roulette spins applying double strategy:

In [ ]:
# Simulate 10000 experiments of 100 runs
df_experiments = simulate(10000,100)

100%|██████████| 10000/10000 [00:52<00:00, 191.85it/s]


In [ ]:
# Simulate 10000 experiments of 100 runs quiting at 50% if profit
df_experiments_pct = simulate(10000,100, 0.5)

100%|██████████| 10000/10000 [00:48<00:00, 205.63it/s]


In [ ]:
df_experiments

,Round_num,WinR,WinB,MaxBetR,MaxBetB
0,0,52,42,32,32
1,1,36,56,2048,16
2,2,51,43,64,128
3,3,58,37,16,128
4,4,49,32,128,128
...,...,...,...,...,...
9995,9995,57,39,128,256
9996,9996,42,49,16,16
9997,9997,49,48,32,256
9998,9998,47,47,64,32


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
df_experiments_pct

,Round_num,WinR,WinB,MaxBetR,MaxBetB
0,0,30,13,8,64
1,1,21,26,16,8
2,2,23,26,32,256
3,3,24,24,64,16
4,4,24,24,32,32
...,...,...,...,...,...
9995,9995,23,27,64,32
9996,9996,4,30,32,16
9997,9997,16,28,16,16
9998,9998,28,20,8,256


In above tables (ordered from highest to lowest amount bet in Black) we see numbers like 131072 or 2097152 as the money one have to bet considering the color you are betting against appears consecutively in your runs. So how often does it usually happen? Lets consider a gambler with 127€ in his/her wallet, that gambler would be able to survive just 6 consecutive spins of the roulette. We now observe how many rounds (of length 100) of a total of 10,000 showed 6 consecutive colors or more:

In [ ]:
(2**7)-1

127

In [ ]:
def count_consecutives(n_exp,max_num, bet_against):
  cont = 0
  for i in tqdm(range(n_exp)):
    if (i > 10):
      break
    ls_runs = [random.choice(np.arange(0,3), p=[0.02702, 0.48649, 0.48649]) for _ in range(100)]
    cont_seqs = [[k,len(list(v))] for k,v in groupby(ls_runs)]
    cont_seqs = [x for x in cont_seqs if x[0]!=bet_against]
    print(cont_seqs)
    for x in cont_seqs:
      if x[1]>max_num:
        cont+=1
        break
  return (cont/n_exp)*100, cont

In [ ]:
# 10 thousand gamblers betting to 2 so we count how many rounds show consecutive series of ones
count_consecutives(10000,12,1)

  0%|          | 11/10000 [00:00<00:32, 310.19it/s]

[[0, 1], [2, 2], [2, 1], [2, 1], [2, 2], [2, 2], [2, 1], [2, 1], [2, 1], [2, 1], [2, 2], [2, 2], [2, 10], [0, 1], [2, 2], [2, 1], [2, 2], [0, 1], [2, 4], [2, 1], [2, 1], [2, 1], [0, 1], [2, 1], [2, 2], [2, 4], [2, 2], [2, 1], [2, 1]]
[[2, 1], [2, 1], [2, 2], [2, 3], [2, 1], [2, 1], [0, 1], [2, 1], [2, 1], [2, 1], [2, 2], [2, 2], [2, 2], [2, 1], [2, 6], [2, 1], [2, 5], [2, 2], [2, 1], [0, 1], [2, 1], [0, 1], [2, 1], [2, 2], [2, 1], [2, 1], [2, 3], [2, 1], [2, 2]]
[[2, 2], [2, 1], [2, 3], [2, 1], [2, 1], [2, 3], [2, 2], [2, 2], [2, 5], [2, 3], [2, 1], [2, 2], [2, 1], [2, 1], [2, 4], [2, 6], [2, 3], [2, 1], [2, 3], [2, 1], [2, 2], [2, 1], [2, 3], [2, 2], [2, 2]]
[[2, 4], [2, 2], [2, 1], [2, 8], [2, 1], [2, 5], [0, 1], [2, 9], [2, 2], [2, 1], [2, 7], [2, 1], [2, 2], [0, 1], [2, 1], [2, 1], [2, 1], [2, 1], [2, 4], [2, 1], [2, 1], [2, 1], [2, 1]]
[[2, 4], [2, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 1], [2, 2], [2, 2], [2, 2], [2, 1], [2, 3], [2, 1], [2, 3], [2, 1], [2, 3], [2, 4], [2, 2], [2

(0.0, 0)

27.59% of those 10.000 rounds showed to have at least one sequence of ones of a length more than 6.

# Others:

In [ ]:
# Percentage of winner rounds for both colors and spent less or equal than 64€ for a player with 127€ in wallet 
len(df_experiments[(df_experiments['WinR']>0)&(df_experiments['WinB']>0)&((df_experiments['MaxBetR']<=64)&(df_experiments['MaxBetB']<=64))])/len(df_experiments)*100

39.39

In [ ]:
# Same question but for double percentage strategy
len(df_experiments_pct[(df_experiments_pct['WinR']>0)&(df_experiments_pct['WinB']>0)&((df_experiments_pct['MaxBetR']<=64)&(df_experiments_pct['MaxBetB']<=64))])/len(df_experiments)*100

62.480000000000004

Looks like you double your chance of winning if you quit earlier. Being more realistic if we are willing to spend 127€ we would want to make at least a 20% profit, lets see for both strategies how many rounds obtained at least 20% in profit:

In [ ]:
127*0.2

25.400000000000002

In [ ]:
len(df_experiments[(df_experiments['WinR']>=25)&(df_experiments['WinB']>=25)&((df_experiments['MaxBetR']<=64)&(df_experiments['MaxBetB']<=64))])/len(df_experiments)*100

37.13

In [ ]:
len(df_experiments_pct[(df_experiments_pct['WinR']>=25)&(df_experiments_pct['WinB']>=25)&((df_experiments_pct['MaxBetR']<=64)&(df_experiments_pct['MaxBetB']<=64))])/len(df_experiments_pct)*100

1.46

In [ ]:
# Number of winner rounds for both colors and spent less than 64€ for a player with 127€ in wallet 
df_experiments[(df_experiments['WinR']>=25)&(df_experiments['WinB']>=25)&((df_experiments['MaxBetR']<64)&(df_experiments['MaxBetB']<64))]

,Round_num,WinR,WinB,MaxBetR,MaxBetB
5,5,47,44,32,32
6,6,52,46,32,32
9,9,46,51,16,32
30,30,43,53,32,32
36,36,44,53,32,16
...,...,...,...,...,...
9976,9976,52,31,16,16
9988,9988,50,48,32,32
9990,9990,52,44,16,32
9993,9993,55,43,16,16


In [ ]:
df_experiments_pct[(df_experiments_pct['WinR']>=25)&(df_experiments_pct['WinB']>=25)&((df_experiments_pct['MaxBetR']<=64)&(df_experiments_pct['MaxBetB']<=64))]

,Round_num,WinR,WinB,MaxBetR,MaxBetB
236,236,25,26,16,64
248,248,25,26,16,64
315,315,25,25,32,32
401,401,25,25,16,32
530,530,25,25,16,32
...,...,...,...,...,...
9720,9720,25,25,16,8
9754,9754,25,25,8,16
9803,9803,25,25,8,16
9817,9817,25,25,16,32
